In [2]:
from collections import namedtuple
from random import choice
import numpy as np
from heapq import heappop, heappush
from tqdm.auto import tqdm







In [3]:
Action = namedtuple('Action', ['from_pos', 'to_pos'])

class PuzzleSolver:
    def __init__(self, puzzle_dim=3, randomize_steps=100_000):
        self.PUZZLE_DIM = puzzle_dim
        self.randomize_steps = randomize_steps
        self.goal_state = self._generate_goal_state()
        self.initial_state = self._randomize_puzzle()

    def _generate_goal_state(self):
        goal = np.arange(1, self.PUZZLE_DIM**2).tolist() + [0]
        return np.array(goal).reshape(self.PUZZLE_DIM, self.PUZZLE_DIM)

    def _available_moves(self, state):
        x, y = np.argwhere(state == 0)[0]
        moves = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.PUZZLE_DIM and 0 <= ny < self.PUZZLE_DIM:
                moves.append(Action((x, y), (nx, ny)))
        return moves

    def _apply_move(self, state, move):
        new_state = state.copy()
        x1, y1 = move.from_pos
        x2, y2 = move.to_pos
        new_state[x1, y1], new_state[x2, y2] = new_state[x2, y2], new_state[x1, y1]
        return new_state

    def _manhattan_heuristic(self, state):
        distance = 0
        for val in range(1, self.PUZZLE_DIM**2):
            current_pos = np.argwhere(state == val)[0]
            goal_pos = np.argwhere(self.goal_state == val)[0]
            distance += abs(current_pos[0] - goal_pos[0]) + abs(current_pos[1] - goal_pos[1])
        return distance

    def _randomize_puzzle(self):
        state = self.goal_state.copy()
        for _ in tqdm(range(self.randomize_steps), desc="Randomizing"):
            move = choice(self._available_moves(state))
            state = self._apply_move(state, move)
        return state

    def solve_with_a_star(self):
      frontier = [(self._manhattan_heuristic(self.initial_state), 0, tuple(self.initial_state.flatten()), [])]
      visited = set()

      while frontier:
          _, cost, state_tuple, path = heappop(frontier)
          current_state = np.array(state_tuple).reshape(self.PUZZLE_DIM, self.PUZZLE_DIM)

          if state_tuple in visited:
              continue

          visited.add(state_tuple)

          if np.array_equal(current_state, self.goal_state):
              return path

          for move in self._available_moves(current_state):
              new_state = self._apply_move(current_state, move)
              new_state_tuple = tuple(new_state.flatten())
              if new_state_tuple not in visited:
                  new_cost = cost + 1
                  new_path = path + [move]
                  estimated_cost = new_cost + self._manhattan_heuristic(new_state)
                  heappush(frontier, (estimated_cost, new_cost, new_state_tuple, new_path))

      return None  # No solution found

    def print_solution(self, solution):
        if not solution:
            print("No solution found.")
        else:
            print(f"Number of steps: {len(solution)}")
            print("Solution moves:")
            for step in solution:
                print(f"Move from {step.from_pos} to {step.to_pos}")

In [8]:
if __name__ == "__main__":
    # Initialize the solver
    solver = PuzzleSolver(puzzle_dim=3, randomize_steps=50_000)

    # Solve the puzzle
    solution = solver.solve_with_a_star()

    # Display results
    solver.print_solution(solution)

Randomizing:   0%|          | 0/50000 [00:00<?, ?it/s]

Number of steps: 20
Solution moves:
Move from (0, 2) to (1, 2)
Move from (1, 2) to (2, 2)
Move from (2, 2) to (2, 1)
Move from (2, 1) to (2, 0)
Move from (2, 0) to (1, 0)
Move from (1, 0) to (1, 1)
Move from (1, 1) to (0, 1)
Move from (0, 1) to (0, 2)
Move from (0, 2) to (1, 2)
Move from (1, 2) to (1, 1)
Move from (1, 1) to (0, 1)
Move from (0, 1) to (0, 0)
Move from (0, 0) to (1, 0)
Move from (1, 0) to (1, 1)
Move from (1, 1) to (1, 2)
Move from (1, 2) to (0, 2)
Move from (0, 2) to (0, 1)
Move from (0, 1) to (1, 1)
Move from (1, 1) to (1, 2)
Move from (1, 2) to (2, 2)
